In [1]:
import ee
import geemap
import os
import calendar

In [2]:
ee.Initialize()

In [3]:
ASSET_ID = 'COPERNICUS/S1_GRD'

roi = ee.Geometry.Polygon(
        [[[36.050662943101294, -0.30239147953987083],
          [36.050662943101294, -0.42323864359236807],
          [36.12447733519114, -0.42323864359236807],
          [36.12447733519114, -0.30239147953987083]]]);

In [4]:
collection = ee.ImageCollection(ASSET_ID)

Map = geemap.Map()

In [5]:
def fmask(img):
    edge = img.lt(-30.0)
    maskedImage = img.mask().And(edge.Not());
    return img.updateMask(maskedImage);    

In [6]:
def prepFilter(coll):
    return(coll
           .filterBounds(roi)
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
           .filter(ee.Filter.eq('instrumentMode', 'IW'))
           .select('VV')
           .map(fmask)
       )

In [7]:
def get_start_end_dates(year,months):
    MONTHS = []
    
    for month in months:
        _,no_of_days = calendar.monthrange(YEAR, month)
        month_str = str(month)
        month_start = f'{YEAR}-{month_str.zfill(2)}-01'
        month_end = f'{YEAR}-{month_str.zfill(2)}-{no_of_days}'
        month_name = calendar.month_name[month]
        MONTHS.append((month_start,month_end,month_name))
        
    return MONTHS

In [8]:
def filter_date(coll,dates):
    start_date,end_date,_ = dates
    return coll.filterDate(start_date,end_date)


def createImageCollectionFromMonths(coll, months):
    images = []
    
    for month in months:
        image_collection = filter_date(coll, month)
        image = image_collection.mean()
        images.append(image)

    return ee.ImageCollection.fromImages(images)


In [9]:
collection = prepFilter(collection)

YEAR = 2020
MONTHS = [1,5,9]

months = get_start_end_dates(YEAR,MONTHS)

newCollection = createImageCollectionFromMonths(collection,months)

month_names = [f'{month[2]} - {YEAR}' for month in months]

In [10]:
# Map.addLayer(newCollection.first().clip(roi),{"min":-25,"max":5})
# Map

In [11]:
video_args = {
  'dimensions': 600,
  'region': roi,
  'framesPerSecond': 1,
  'bands': ['VV',],
  'min': -25, 
  'max': 5,
  'gamma':1
}

work_dir = os.path.join(os.path.expanduser("~"), 'Downloads')
out_gif = os.path.join(work_dir, "test.gif")

geemap.download_ee_video(newCollection, video_args, out_gif)

texted_gif = os.path.join(work_dir, "test_text.gif")

geemap.add_text_to_gif(out_gif,
                       texted_gif, 
                       xy=('3%', '5%'), 
                       font_size=30, 
                       font_color='#ffffff', 
                       duration=1000,
                       text_sequence = month_names,
                       add_progress_bar=True)

geemap.show_image(texted_gif)

Generating URL...
Please wait ...
The GIF image has been saved to: /Users/erickotenyo/Downloads/test.gif


Output()